In [1]:
import itertools
import json


import numpy
import pandas
import rich

In [12]:
data = json.load(open("../../../data/moral_foundations/surveys.json"))["graham_et_al"]
rich.print(data)

{
    'anonymous': {
        'liberal': {'authority': 2.2, 'purity': 2.2, 'ingroup': 2.5, 'harm': 4.25, 'fairness': 4.35},
        'moderate': {'authority': 2.8, 'purity': 3.0, 'ingroup': 3.0, 'harm': 4.0, 'fairness': 3.95},
        'conservative': {'authority': 3.1, 'purity': 3.4, 'ingroup': 3.12, 'harm': 3.5, 'fairness': 3.65}
    },
    'U.S.': {
        'liberal': {'authority': 1.9, 'purity': 1.2, 'ingroup': 2.05, 'harm': 3.6, 'fairness': 3.75},
        'moderate': {'authority': 2.65, 'purity': 2.2, 'ingroup': 2.65, 'harm': 3.4, 'fairness': 3.45},
        'conservative': {'authority': 3.5, 'purity': 3.1, 'ingroup': 3.4, 'harm': 3.0, 'fairness': 3.1}
    },
    'Korean': {
        'liberal': {'authority': 2.25, 'purity': 2.4, 'ingroup': 2.3, 'harm': 3.35, 'fairness': 3.5},
        'moderate': {'authority': 2.55, 'purity': 2.65, 'ingroup': 2.65, 'harm': 3.25, 'fairness': 3.35},
        'conservative': {'authority': 2.9, 'purity': 2.75, 'ingroup': 2.8, 'harm': 3.35, 'fairness': 3.1}
    }
}

In [13]:
records = [
    (country, ideology, values)
    for country, ideologies in data.items()
    for ideology, values in ideologies.items()
]

In [14]:
df = pandas.DataFrame(
    [values for _, _, values in records],
    index=pandas.MultiIndex.from_tuples(
        [(c, i) for c, i, _ in records], names=["country", "ideology"]
    ),
)
df.head()

authority  purity  ingroup  harm  fairness
country   ideology                                                
anonymous liberal            2.20     2.2     2.50  4.25      4.35
          moderate           2.80     3.0     3.00  4.00      3.95
          conservative       3.10     3.4     3.12  3.50      3.65
U.S.      liberal            1.90     1.2     2.05  3.60      3.75
          moderate           2.65     2.2     2.65  3.40      3.45

In [15]:
mse_df = (
    pandas.DataFrame.from_dict(
        {
            (idx1, idx2): numpy.mean(abs(numpy.subtract(df.loc[idx1], df.loc[idx2])))
            for idx1, idx2 in list(itertools.combinations(df.index, 2))
        }, 
        orient="index", 
        columns=["MSE"]
    )
    .sort_values("MSE")
)
mse_df.head()

,MSE
"((U.S., moderate), (Korean, moderate))",0.16
"((Korean, moderate), (Korean, conservative))",0.19
"((U.S., moderate), (Korean, liberal))",0.21
"((Korean, liberal), (Korean, moderate))",0.23
"((U.S., moderate), (Korean, conservative))",0.27


In [16]:
mse_df.min(), mse_df.max()

(MSE    0.16
 dtype: float64,
 MSE    1.22
 dtype: float64)

In [17]:
mse_matrix = pandas.DataFrame(index=df.index, columns=df.index)

for (idx1, idx2), mse_value in {
    (idx1, idx2): numpy.mean(abs(numpy.subtract(df.loc[idx1], df.loc[idx2])))
    for idx1, idx2 in list(itertools.combinations(df.index, 2))
}.items():
    mse_matrix.loc[idx1, idx2] = mse_value
    mse_matrix.loc[idx2, idx1] = mse_value

mse_matrix = mse_matrix.astype(float)
mse_matrix

country                anonymous                          U.S.           \
ideology                 liberal moderate conservative liberal moderate   
country   ideology                                                        
anonymous liberal            NaN    0.510        0.834   0.600    0.470   
          moderate         0.510      NaN        0.324   0.850    0.480   
          conservative     0.834    0.324          NaN   0.934    0.484   
U.S.      liberal          0.600    0.850        0.934     NaN    0.570   
          moderate         0.470    0.480        0.484   0.570      NaN   
          conservative     1.120    0.610        0.406   1.220    0.650   
Korean    liberal          0.440    0.590        0.594   0.460    0.210   
          moderate         0.590    0.460        0.464   0.690    0.160   
          conservative     0.740    0.410        0.374   0.840    0.270   

country                              Korean                        
ideology               conservative liberal moderate conservative  
country   ideology                                                 
anonymous liberal             1.120   0.440    0.590        0.740  
          moderate            0.610   0.590    0.460        0.410  
          conservative        0.406   0.594    0.464        0.374  
U.S.      liberal             1.220   0.460    0.690        0.840  
          moderate            0.650   0.210    0.160        0.270  
          conservative          NaN   0.760    0.530        0.380  
Korean    liberal             0.760     NaN    0.230        0.380  
          moderate            0.530   0.230      NaN        0.190  
          conservative        0.380   0.380    0.190          NaN